#### Notes

This code takes a main folder with subsubsolders where the labels per frame are stored. The folder tree looks, for example, like:

Sweden:\
-Channel_1\
--Video_1\
---labels\
----label_1.txt\
----label_2.txt\
----...\
--Video_2\
---labels\
----label_1.txt\
...

* Some of the labels directories are empty 

In [1]:
import os
import pandas as pd
#import parquet 
from collections import defaultdict
import os.path
from fastparquet import write

SystemError: initialization of _internal failed without raising an exception

In [ ]:
%conda install 

In [ ]:
def result_processing(path_to_results_dir:str, path_to_res_file:str, country:str):
    ### This will return a parque file 
    
    # File to store the results
    file_path = path_to_res_file+f'results_{country}.parquet'
    
    # Channel names 
    channel_dirs = [filename for filename in os.listdir(path_to_results_dir) if not filename.startswith('.')]
    for channel_d in channel_dirs:
        sub_folder_path = os.path.join(path_to_results_dir,channel_d)
        # Access the video's frames folder 
        videos_dir = [filename for filename in os.listdir(sub_folder_path) if not filename.startswith('.')]
        for video_d in videos_dir:
            video_path = os.path.join(sub_folder_path,video_d)
            labels_dir_path = os.path.join(video_path,'labels/')
            
            if os.listdir(labels_dir_path): # Labels dir is not empty:
                
                # Access labels information. This are txt files
                labels_files = [filename for filename in os.listdir(labels_dir_path) if not filename.startswith('.')]
                for label_f in labels_files:
                    
                    with open(os.path.join(labels_dir_path,label_f), 'r') as f:
                        lines = f.readlines()
                        # Write the labels to a data frame
                        df = pd.DataFrame([l.split() for l in lines],columns=['class_label',
                                'x_center','y_center','width','height','confidence_level'])
                        df.insert(0,'Frame', label_f.split('.')[0])
                        df.insert(0, 'Video_id', video_d)
                        df.insert(0,'Channel', channel_d)
       
                        # If the file does not exists, create it
                        if not os.path.isfile(file_path): 
                            write(file_path, df)
                        else: # Otherwise, write on it
                            write(file_path, df, append=True)
            else: 
                pass
            
    return file_path 


def transf_parque(parque_file_path:str):
    res = pd.read_parquet(parque_file_path, engine='fastparquet')
    res
    # Converting to CSV
    res.to_csv(parque_file_path.split('.par')[0]+'.csv', index = False)

In [87]:
main_folder = '../../../yolov5/Sweden_analysis_TEST'
end_results_folder = '../../data_collection/data/'
country = 'sweden'

In [88]:
res = result_processing(main_folder, end_results_folder, country)
transf_parque(res)

In [90]:
res = pd.read_parquet('../../data_collection/data/results_sweden.parquet', engine='fastparquet')

In [105]:
res.head()

,Channel,Video_id,Frame,class_label,x_center,y_center,width,height,confidence_level
0,Skolledarforbundet,TWQvMVtFPjc,TWQvMVtFPjc_2700,1,0.507611,0.375,0.165105,0.258333,0.755785
1,Skolledarforbundet,nkm5M0DpwhI,nkm5M0DpwhI_1750,1,0.471311,0.823958,0.0222482,0.0395833,0.429938
2,Skolledarforbundet,TKPUP6cenBE,TKPUP6cenBE_3500,1,0.608073,0.650463,0.0265625,0.0490741,0.693334
3,Skolledarforbundet,TKPUP6cenBE,TKPUP6cenBE_4000,1,0.638542,0.808796,0.01875,0.0305556,0.475797
4,Skolledarforbundet,TKPUP6cenBE,TKPUP6cenBE_750,1,0.449479,0.667593,0.0270833,0.0518519,0.659074


In [92]:
res.shape

(18346, 9)

In [114]:
res = res.astype({'confidence_level':'float'})

In [115]:
res[res['confidence_level'] > 0.85]

,Channel,Video_id,Frame,class_label,x_center,y_center,width,height,confidence_level
480,handelsanstalldasforbund,f_7slw419Lo,f_7slw419Lo_870,1,0.172266,0.11875,0.0992187,0.1625,0.899152
483,handelsanstalldasforbund,f_7slw419Lo,f_7slw419Lo_290,1,0.172266,0.122222,0.102344,0.169444,0.904200
484,handelsanstalldasforbund,f_7slw419Lo,f_7slw419Lo_580,1,0.172266,0.118056,0.0992187,0.161111,0.897128
486,handelsanstalldasforbund,f_7slw419Lo,f_7slw419Lo_1160,1,0.172266,0.118056,0.0992187,0.161111,0.899717
524,handelsanstalldasforbund,zw4_igN8xns,zw4_igN8xns_6750,1,0.953125,0.356944,0.0895833,0.213889,0.930246
...,...,...,...,...,...,...,...,...,...
18264,losverige,wqLABHlwXt8,wqLABHlwXt8_3250,1,0.180729,0.11713,0.0822917,0.150926,0.927944
18266,losverige,wqLABHlwXt8,wqLABHlwXt8_750,1,0.18125,0.116667,0.0833333,0.15,0.931602
18267,losverige,wqLABHlwXt8,wqLABHlwXt8_6500,1,0.18151,0.115278,0.0828125,0.149074,0.937273
18268,losverige,wqLABHlwXt8,wqLABHlwXt8_1000,1,0.18099,0.116204,0.0828125,0.150926,0.933910


In [116]:
res_csv = pd.read_csv('../../data_collection/data/results_sweden.csv')

In [117]:
res_csv.head()

,Channel,Video_id,Frame,class_label,x_center,y_center,width,height,confidence_level
0,Skolledarforbundet,TWQvMVtFPjc,TWQvMVtFPjc_2700,1,0.507611,0.375000,0.165105,0.258333,0.755785
1,Skolledarforbundet,nkm5M0DpwhI,nkm5M0DpwhI_1750,1,0.471311,0.823958,0.022248,0.039583,0.429938
2,Skolledarforbundet,TKPUP6cenBE,TKPUP6cenBE_3500,1,0.608073,0.650463,0.026562,0.049074,0.693334
3,Skolledarforbundet,TKPUP6cenBE,TKPUP6cenBE_4000,1,0.638542,0.808796,0.018750,0.030556,0.475797
4,Skolledarforbundet,TKPUP6cenBE,TKPUP6cenBE_750,1,0.449479,0.667593,0.027083,0.051852,0.659074


In [118]:
res_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18346 entries, 0 to 18345
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Channel           18346 non-null  object 
 1   Video_id          18346 non-null  object 
 2   Frame             18346 non-null  object 
 3   class_label       18346 non-null  int64  
 4   x_center          18346 non-null  float64
 5   y_center          18346 non-null  float64
 6   width             18346 non-null  float64
 7   height            18346 non-null  float64
 8   confidence_level  18346 non-null  float64
dtypes: float64(5), int64(1), object(3)
memory usage: 1.3+ MB
